In [ ]:
# 9월 28일 정리

In [6]:
# %reset          # 메모리 초기화

# Logistic Regression을 python, tensorflow, sklearn으로 각각 구현
# 처음은 독립변수가 1개인 걸로 하자

import numpy as np
import tensorflow as tf
from sklearn import linear_model

# 수치미분함수 들고와서 쓰자
###########################
def numerical_derivative(f,x):
    # f : 미분하려고 하는 다변수 함수
    # x : 모든 변수를 포함하고 있는 ndarray (차원 상관없이)
    
    delta_x = 1e-4
    derivative_x = np.zeros_like(x)  # 미분한 결과를 저장하는 ndarray
    
    # iterator를 이용해서 입력변수 x에 대해 편미분을 수행
    it = np.nditer(x, flags=['multi_index'])
    
    while not it.finished:
        idx = it.multi_index    # iterator의 현재 index를 추출 (tuple)
        
        # 현재 칸의 값을 어딘가에 잠시 저장해야한다.
        tmp = x[idx]
        
        x[idx] = tmp + delta_x
        fx_plus_delta = f(x)   # f(x + delta_x)
        
        x[idx] = tmp - delta_x
        fx_minus_delta = f(x)   # f(x - delta_x)
        
        derivative_x[idx] = (fx_plus_delta - fx_minus_delta) / (2 * delta_x)
        
        x[idx] = tmp     # 데이터를 원상 복구
        
        it.iternext()
        
    return derivative_x

##########################
# Raw Data Loading + Data Preprocessing
# 이번 예제에는 이 과정 필요 없음

# Training Data Set
# 지도학습을 하고 있기 때문에 독립변수와 종속변수 (label)로 구분해서 데이터 준비
# 어떤 경우에는 이 두객를 아예 분리해서 제공하는 경우도 있다. 참고

x_data = np.arange(2,21,2).reshape(-1,1)
t_data = np.array([0,0,0,0,0,0,1,1,1,1])

###########################
# python 구현부터 해보자

# Weight & bias
W = np.random.rand(1,1)
b = np.random.rand(1)

# 위에서 정의한 W와 b의 값을 구해야 한다.
# 이 값만 구하면 우리의 최종 목적인 model을 완성할 수 있다.

# loss function(손실함수) - 우리 모델의 예측값과 들어온 t_data(정답)
# 입력으로 들어온 x_data와 W,b 값을 이용해 예측값 계산, t_data(정답)을 비교함
def loss_func(input_obj):     # input_obj : W와 b를 같이 포함하는 ndarray [W1 W2 b]
    
    num_of_bias = b.shape[0]   # num_of_bias : 1
    input_W = input_obj[:-1*num_of_bias].reshape(-1,num_of_bias)  # W 생성
    input_b = input_obj[-1*num_of_bias]
    
    # 우리 모델의 예측값은 : (linear regression model(Wx+b) => sigmoid를 적용)
    z = np.dot(x_data, input_W) + input_b
    y = 1 / ( 1 + np.exp(-1 * z) )
    
    delta = 1e-7 # 굉장히 작은값을 이용해서 프로그램으로 로그연산시 
                 # 무한대로 발산하는 것 방지
    
    # cross entropy
    return -np.sum(t_data*np.log(y+delta) + ((1-t_data)*np.log(1-y+delta)))

# learning rate
learning_rate = 1e-4

# 학습
for step in range(30000):
    input_param = np.concatenate((W.ravel(), b.ravel()), axis=0)  # [W1 W2 b]
    derivative_result = learning_rate * numerical_derivative(loss_func, input_param)
    
    num_of_bias = b.shape[0]
    
    W = W - derivative_result[:-1*num_of_bias].reshape(-1,num_of_bias) # [[W1] [W2]]
    b = b - derivative_result[-1*num_of_bias:]
    
# predict => W,b를 다 구해서 우리의 logistic Regression model을 완성
def logistic_predict(x):   # 공부한 시간이 입력
    z = np.dot(x,W) + b
    y = 1 / ( 1 + np.exp(-1*z) )
    
    if y < 0.5 :
        result = 0
    else:
        result = 1
        
    return result, y

study_hour = np.array([[13]])
result = logistic_predict(study_hour)
print('######## python 결과값 ##########')
print('공부시간 : {}, 결과 : {}'.format(study_hour,result))

######## python 결과값 ##########
공부시간 : [[13]], 결과 : (0, array([[0.39999997]]))


In [9]:
### sklearn으로 구현해보자

# logistic regression model 생성
model = linear_model.LogisticRegression()

# training data set을 이용해서 학습
model.fit(x_data,t_data.ravel())       # x는 2차원, label은 1차원 vector로 넣어야함

study_hour = np.array([[13]])
predict_val = model.predict(study_hour)
predict_prob = model.predict_proba(study_hour)

print('######## sklearn 결과값 ##########')
print('공부시간 : {}, 결과 : {},{}'.format(study_hour,predict_val,predict_prob))

######## sklearn 결과값 ##########
공부시간 : [[13]], 결과 : [0],[[0.50009391 0.49990609]]


In [16]:
# Tensorflow

# placeholder
X = tf.placeholder(dtype=tf.float32) # 독립변수가 1개인 경우 shape을 명시하지 않음
T = tf.placeholder(dtype=tf.float32) # t_data

# Weight & bias
W = tf.Variable(tf.random.normal([1,1]), name='weight')
W = tf.Variable(tf.random.normal([1]), name='bias')

# Hypothesis
logit = W * X + b  # matrix 곱연산 하지 않나요 ??
H = tf.sigmoid(logit)

# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-3).minimize(loss)

# session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(30000):
    sess.run(train, feed_dict={X:x_data, T:t_data})
    
study_hour = np.array([13])
result = sess.run(H,feed_dict={X:study_hour})
print('######## tensorflow 결과값 ##########')
print('공부시간 : {}, 결과 : {}'.format(study_hour,result))

######## tensorflow 결과값 ##########
공부시간 : [13], 결과 : [0.40000007]


In [36]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn import linear_model
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt 

# Multi Variable Logistic Regression
# 독립변수가 2개 이상인 logistic regression

# 학습하는 데이터는 GRE(Graduate Record Examination)와
# GPA 성적 그리고 Rank에 대한 대학원 합격/불합격 정보

# 내 성적 [600. 3.8 1.] 의 결과
# 첫번째는 sklearn으로 구현하기

# Raw Data Loading 
df = pd.read_csv('./data/admission/admission.csv')

# 결측치 확인
print(df.isnull().sum())     # 결측치가 없음 -> dropna 안해도됨

# 이상치를 확인해서 있으면 제거
# fig = plt.figure()
# fig_admit = fig.add_subplot(1,4,1)
# fig_gre = fig.add_subplot(1,4,2)
# fig_gpa = fig.add_subplot(1,4,3)
# fig_rank = fig.add_subplot(1,4,4)

# fig_admit.boxplot(df['admit'])
# fig_gre.boxplot(df['gre'])
# fig_gpa.boxplot(df['gpa'])
# fig_rank.boxplot(df['rank'])

# fig.tight_layout()
# plt.show()

# 확인했더니 이상치가 있어서 제거하려한다.
zscore_threshold = 2.0

for col in df.columns:
    outlier = df[col][np.abs(stats.zscore(df[col])) > zscore_threshold]
    df = df[~df[col].isin(outlier)]
    
print(df.shape)        # (382, 4)

# Training Data Set
x_data = df.drop('admit', axis=1, inplace=False).values
t_data = df['admit'].values.reshape(-1,1)


# 정규화를 진행해야한다.
scaler_x = MinMaxScaler()
scaler_x.fit(x_data)
norm_x_data = scaler_x.transform(x_data)   # for python, tensorflow

# sklearn을 이용한 구현
model = linear_model.LogisticRegression()
model.fit(x_data,t_data.ravel())
print('###### sklearn 결과 ######')
my_score = np.array([[600, 3.8, 1]])
predict_val = model.predict(my_score)  # 0 or 1
predict_proba = model.predict_proba(my_score)  # 불합격/합격 확률
print(my_score, predict_val, predict_proba)


# Tensorflow

# placeholder
X = tf.placeholder(shape=[None, 3], dtype=tf.float32)  # 독립변수의 데이터를 받기위한 placeholder
T = tf.placeholder(shape=[None, 1], dtype=tf.float32)  # 종속변수(label)의 placeholder

# Weight & bias
W = tf.Variable(tf.random.normal([3,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

# hypothesis
logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-3).minimize(loss)

# session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(30000):
    _, W_val, b_val, loss_val = sess.run([train,W,b,loss], 
                                         feed_dict={X:norm_x_data, T:t_data})
    
    if step % 3000 == 0:
        print('W:{},b:{},loss:{}'.format(W_val,b_val,loss_val))
        
my_score = np.array([[600,3.8,1]])
scaled_my_score = scaler_x.transform(my_score)

result = sess.run(H,feed_dict={X:scaled_my_score})
print('######## tensorflow 결과값 ##########')
print('점수 : {}, 결과 : {}'.format(my_score,result))

admit    0
gre      0
gpa      0
rank     0
dtype: int64
(382, 4)
###### sklearn 결과 ######
[[600.    3.8   1. ]] [1] [[0.43740782 0.56259218]]
W:[[ 0.70904505]
 [-0.14659591]
 [-0.09499577]],b:[-0.53022546],loss:0.6417478919029236
W:[[ 0.6460832 ]
 [-0.1878663 ]
 [-0.27333695]],b:[-0.71348506],loss:0.6152343153953552
W:[[ 0.65193063]
 [-0.15737596]
 [-0.38120386]],b:[-0.7708575],loss:0.6097505688667297
W:[[ 0.6744258 ]
 [-0.11012489]
 [-0.4665115 ]],b:[-0.7947191],loss:0.6062067151069641
W:[[ 0.7000656 ]
 [-0.06026416]
 [-0.5418601 ]],b:[-0.80907273],loss:0.6031953692436218
W:[[ 0.72520995]
 [-0.01160642]
 [-0.6109964 ]],b:[-0.8205985],loss:0.6005567312240601
W:[[ 0.7489259 ]
 [ 0.03488902]
 [-0.67522347]],b:[-0.83119816],loss:0.5982351899147034
W:[[ 0.7710455 ]
 [ 0.07906872]
 [-0.735134  ]],b:[-0.84141755],loss:0.5961896777153015
W:[[ 0.79161364]
 [ 0.1210006 ]
 [-0.79110485]],b:[-0.8513822],loss:0.5943844318389893
W:[[ 0.810732  ]
 [ 0.16080914]
 [-0.8434311 ]],b:[-0.861151],loss:0.

In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn import linear_model
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt 

# Multi Variable Logistic Regression
# 독립변수가 2개 이상인 logistic regression

# 학습하는 데이터는 GRE(Graduate Record Examination)와
# GPA 성적 그리고 Rank에 대한 대학원 합격/불합격 정보

# 내 성적 [600. 3.8 1.] 의 결과
# 첫번째는 sklearn으로 구현하기

# Raw Data Loading 
df = pd.read_csv('./data/admission/admission.csv')

# 결측치 확인
print(df.isnull().sum())     # 결측치가 없음 -> dropna 안해도됨

# 이상치를 확인해서 있으면 제거
# fig = plt.figure()
# fig_admit = fig.add_subplot(1,4,1)
# fig_gre = fig.add_subplot(1,4,2)
# fig_gpa = fig.add_subplot(1,4,3)
# fig_rank = fig.add_subplot(1,4,4)

# fig_admit.boxplot(df['admit'])
# fig_gre.boxplot(df['gre'])
# fig_gpa.boxplot(df['gpa'])
# fig_rank.boxplot(df['rank'])

# fig.tight_layout()
# plt.show()

# 확인했더니 이상치가 있어서 제거하려한다.
zscore_threshold = 2.0

for col in df.columns:
    outlier = df[col][np.abs(stats.zscore(df[col])) > zscore_threshold]
    df = df[~df[col].isin(outlier)]
    
print(df.shape)        # (382, 4)

# Training Data Set
x_data = df.drop('admit', axis=1, inplace=False).values
t_data = df['admit'].values.reshape(-1,1)


# 정규화를 진행해야한다.
scaler_x = MinMaxScaler()
scaler_x.fit(x_data)
norm_x_data = scaler_x.transform(x_data)   # for python, tensorflow

# Tensorflow

# placeholder
X = tf.placeholder(shape=[None, 3], dtype=tf.float32)  # 독립변수의 데이터를 받기위한 placeholder
T = tf.placeholder(shape=[None, 1], dtype=tf.float32)  # 종속변수(label)의 placeholder

# Weight & bias
W = tf.Variable(tf.random.normal([3,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

# hypothesis
logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-3).minimize(loss)

# session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(30000):
    _, W_val, b_val, loss_val = sess.run([train,W,b,loss], 
                                         feed_dict={X:norm_x_data, T:t_data})
    
    if step % 3000 == 0:
        print('W:{},b:{},loss:{}'.format(W_val,b_val,loss_val))

# 정확도(Accuracy) 측정
predict = tf.cast(H >= 0.5, dtype=tf.float32) # True =>1, False => 0
correct = tf.equal(predict, T)
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

accuracy_val = sess.run(accuracy, feed_dict={X:norm_x_data, T:t_data})
print(accuracy_val)
        
my_score = np.array([[600,3.8,1]])
scaled_my_score = scaler_x.transform(my_score)

result = sess.run(H,feed_dict={X:scaled_my_score})
print('######## tensorflow 결과값 ##########')
print('점수 : {}, 결과 : {}'.format(my_score,result))

admit    0
gre      0
gpa      0
rank     0
dtype: int64
(382, 4)
W:[[ 1.7646356 ]
 [ 0.08762144]
 [-0.2812959 ]],b:[-0.68270737],loss:0.7055816650390625
W:[[ 1.5603307 ]
 [-0.07895922]
 [-0.5250376 ]],b:[-1.0438235],loss:0.6067104339599609
W:[[ 1.5061036 ]
 [-0.08941314]
 [-0.6315138 ]],b:[-1.1422116],loss:0.5978860855102539
W:[[ 1.4913461 ]
 [-0.06034793]
 [-0.7001493 ]],b:[-1.1717452],loss:0.595608651638031
W:[[ 1.4872788 ]
 [-0.02153197]
 [-0.7557979 ]],b:[-1.1817459],loss:0.5940296053886414
W:[[ 1.4858648 ]
 [ 0.01877024]
 [-0.8054961 ]],b:[-1.1860178],loss:0.5926572680473328
W:[[ 1.4847919 ]
 [ 0.05822078]
 [-0.8514294 ]],b:[-1.1885669],loss:0.5914321541786194
W:[[ 1.4833894 ]
 [ 0.09620302]
 [-0.89437455]],b:[-1.1906039],loss:0.5903341770172119
W:[[ 1.4814932 ]
 [ 0.13258183]
 [-0.9346981 ]],b:[-1.192392],loss:0.5893484950065613
W:[[ 1.4791623 ]
 [ 0.16740315]
 [-0.97260696]],b:[-1.1941801],loss:0.5884621739387512
0.68586385
######## tensorflow 결과값 ##########
점수 : [[600.    3.8 